In [ ]:
!pip install boto3 tqdm

In [ ]:
import boto3
import os
from boto3.s3.transfer import TransferConfig
from tqdm import tqdm

In [ ]:
MODEL_REPO = "meta-llama/Meta-Llama-3.1-8B-Instruct"
directory = '/opt/app-root/src/models/models--meta-llama--Meta-Llama-3.1-8B-Instruct/snapshots/8c22764a7e3675c50d4c7c9a4edb474456022b16'
key_base = f'models/{MODEL_REPO}'
bucket_name = os.getenv("AWS_S3_BUCKET")

In [ ]:
config = TransferConfig(multipart_threshold=1024 * 25,  # 25MB
                        max_concurrency=10,
                        multipart_chunksize=1024 * 25,  # 25MB
                        use_threads=True)

In [ ]:
for root, dirs, files in os.walk(directory):
    for file in files:
        file_path = os.path.join(root, file)
        key_name = os.path.join(key_base, file)
        print(key_name)
        # Upload the file
        try:
            with tqdm(total=os.path.getsize(file_path), unit='B', unit_scale=True, desc=file_path) as pbar:
                s3_client = boto3.client('s3')
                s3_client.upload_file(
                    file_path, 
                    bucket_name, 
                    key_name, 
                    Config=config, 
                    Callback=lambda bytes_transferred: pbar.update(bytes_transferred)
                )
            print(f'File {file_path} uploaded to {bucket_name}/{key_name}')
        except Exception as e:
            print(f'Error occurred while uploading {file_path}: {e}')